In [10]:
%matplotlib inline
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers, models, initializers, Model, losses, optimizers
from tensorflow.keras.losses import Loss
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.mixed_precision import experimental as mixed_precision
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import cv2 as cv
from pathlib import Path
from networks import *
from lossFunctions import *
from helperFunctions import *
import datetime
lfCropSize = [375,540,7,7]
angularDim=[7,7]
spatialDim=[375,540]
myDtype = tf.float32
myIntDtype = tf.int32
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [11]:
def padder_cropper(spatialDim,angularDim):
    remainder = 32 - np.array(spatialDim) % 32
    even_odd = remainder % 2
    padder = models.Sequential()
    paddim = [(remainder[0]//2+even_odd[0], remainder[0]//2),(remainder[1]//2+even_odd[1], remainder[1]//2)]
    padder.add(layers.ZeroPadding2D(padding=(paddim[0],paddim[1]),
                                    input_shape=(spatialDim[0],spatialDim[1],3), dtype='float32'))

    cropper_c = models.Sequential()
    cropper_c.add(layers.Cropping2D(cropping=(paddim[0],paddim[1],),
                                  input_shape=(spatialDim[0]+remainder[0],spatialDim[1]+remainder[1],3)
                                  , dtype='float32'))
    cropper_d = models.Sequential()
    cropper_d.add(layers.Cropping3D(cropping=(paddim[0],paddim[1],(0,0)),
                                  input_shape=(spatialDim[0]+remainder[0],spatialDim[1]+remainder[1],angularDim[0],angularDim[1])
                                  , dtype='float32'))
    lf_padder = models.Sequential()
    lf_padder.add(layers.Reshape([spatialDim[0], spatialDim[1], angularDim[0]*angularDim[0],3], input_shape=[spatialDim[0], spatialDim[1], angularDim[0],angularDim[0],3], dtype='float32'))
    lf_padder.add(layers.ZeroPadding3D(padding=(paddim[0],paddim[1],0), dtype='float32'))
    lf_padder.add(layers.Reshape([spatialDim[0]+ paddim[0][0] + paddim[0][1], spatialDim[1] + paddim[1][0] + paddim[1][1], angularDim[0],angularDim[0],3], dtype='float32'))
    return padder,cropper_c, cropper_d,lf_padder,[spatialDim[0]+remainder[0],spatialDim[1]+remainder[1]]

def print_results(step,datasetLength,duration,ETA,loss,metrics,meanMetrics):
    printable = ''
    for key in meanMetrics.keys():
        meanMetrics[key](metrics[key])
    for key in meanMetrics.keys():
        printable += key +': {:.5f} '.format(meanMetrics[key].result().numpy()) + ' '
    print('\r',
        'step {:d}/{:d}: ETA: {:} stepTimeAVG: {:.5f}, Loss {:.6f}'.format(
        step, datasetLength, str(datetime.timedelta(seconds=ETA)) ,duration.result(), loss), printable
        , end='')

In [12]:
from pathlib import Path
@tf.function()
def normalize_lf(lf):
    return 2.0*(lf-0.5)
@tf.function()
def lfRead(name):
    valueLF = tf.io.read_file(name)
    lf = tf.image.decode_png(valueLF, channels=3)
    lf = normalize_lf(tf.image.convert_image_dtype(lf,tf.float32))
    lf = tf.transpose(tf.reshape(lf, [lfCropSize[0], lfCropSize[2], lfCropSize[1], lfCropSize[3], 3]), perm=[0, 2, 1, 3, 4])
    return lf

@tf.function()
def centerRead(name):
    valueLF = tf.io.read_file(name)
    center = tf.image.decode_jpeg(valueLF, channels=3)
    center = normalize_lf(tf.image.convert_image_dtype(center,tf.float32))
    center.set_shape([lfCropSize[0],lfCropSize[1],3])
    return center

@tf.function()
def read(x,y):
    cent = centerRead(x)
    lf = lfRead(y)
    return cent,lf

@tf.function()
def read_center(x,y):
    cent = centerRead(x)
    lf = lfRead(y)
    
    return cent,lf[:,:,3,3,:]

def input_pipeline(Names, center_filenames, lf_filenames,batch_size):
    center_names = tf.data.Dataset.from_tensor_slices(center_filenames)
    lf_names = tf.data.Dataset.from_tensor_slices(lf_filenames)
    cN = tf.data.Dataset.from_tensor_slices(Names)
    names = tf.data.Dataset.zip((center_names,lf_names))
    ds = names.map(read)
    return  tf.data.Dataset.zip((ds,cN)).batch(batch_size, drop_remainder=True)

def center_pipeline(center_filenames, lf_filenames,batch_size):
    center_names = tf.data.Dataset.from_tensor_slices(center_filenames)
    lf_names = tf.data.Dataset.from_tensor_slices(lf_filenames)
    names = tf.data.Dataset.zip((center_names,lf_names))
    ds = names.map(read_center).batch(batch_size, drop_remainder=True)
    
    return ds
edgeP = 5
lf_path_test = '../../../Dataset/30Scenes_7x7/LFPath/'
center_path_test = '../../../Dataset/30Scenes_7x7/centerJpeg/'
train_filenames_lfs_test = [os.path.join(lf_path_test, f) for f in os.listdir(lf_path_test) if f.endswith('.png')]
train_filenames_lfs_test.sort()
train_filenames_centers_test = [os.path.join(center_path_test, f) for f in os.listdir(center_path_test) if f.endswith('center.jpeg')]
train_filenames_centers_test.sort()
imName = []
for cN in train_filenames_centers_test:
    imName.append(Path(cN).stem)
testDataset = input_pipeline(imName,train_filenames_centers_test,train_filenames_lfs_test,1)

In [13]:
padder,centerCropper,depthCropper, lfPadder,spatial_dim = padder_cropper([375,540],angularDim)
# myModel = LFDecompressCompress(spatial_dim,angularDim)
myModel = LFDecompress(spatial_dim,angularDim)
myModel.build(input_shape=[1])

metrics={}
@tf.function(experimental_compile=True)
def step_func(center, lf_Gr):
    cent_Gr = lf_Gr[:,:,:,3,3,:]
    cent = padder(center)
    lf_Gr_pad = lfPadder(lf_Gr)
    center_refined_pad = myModel.jpegHance(cent,training=True)
    ray_depth_pad = myModel.depthEstimation(center_refined_pad)
    center_refined = centerCropper(center_refined_pad)
    center_psnr, center_ssim = center_metrics(cent_Gr,center_refined)
    metrics.update({'center_psnr':center_psnr})
    metrics.update({'center_ssim':center_ssim})
    
    ray_depth = depthCropper(ray_depth_pad)
    lf_lambertian = lambertian_lf_gen(ray_depth, center_refined)
    lam_ssim, lam_psnr = lf_metrics(lf_Gr, lf_lambertian)
    metrics.update({'val_ssim':lam_ssim})
    metrics.update({'val_psnr':lam_psnr})
    
    dof_ssim, dof_psnr = dof_metrics(lf_Gr,lf_lambertian)    
    metrics.update({'dof_ssim':dof_ssim})
    metrics.update({'dof_psnr':dof_psnr})
    
    return metrics

def validate(DS):
    validation_time_spent_average = keras.metrics.Mean()
    testdsLength = tf.data.experimental.cardinality(testDataset).numpy()
    meanMetricsTest = {}
    step = 0
    psnrs = []
    ssims = []
    d_psnrs = []
    d_ssims = []
    for (cents,lf_gt),cNames in DS:
        step+=1
        start = time.time()
        metrics = step_func(cents,lf_gt)
        stop = time.time()
        duration = stop - start
        validation_time_spent_average(duration)
        eta = int((testdsLength - step)*validation_time_spent_average.result())
        psnrs.append(metrics['val_psnr'].numpy())
        ssims.append(metrics['val_ssim'].numpy())
        d_psnrs.append(metrics['dof_psnr'].numpy())
        d_ssims.append(metrics['dof_ssim'].numpy())
        if len(meanMetricsTest)==0:
            for key in metrics.keys():
                meanMetricsTest[key] = keras.metrics.Mean()
        print_results(step,testdsLength,validation_time_spent_average,eta,0.1,metrics,meanMetricsTest)
    return psnrs,ssims, d_psnrs, d_ssims

In [14]:
Allvalidate = []
names = []
for i in (1,):
    print()    
    print(i)
    myModel.load_weights('../../weights/combinedFinalDepth/Large/Lates/2DS/ckpt_'+ str(i))
#     myModel.load_weights('weights/combinedFinalDepth/Large/ckpt_'+str(i))
    Allvalidate.append(validate(testDataset))
    names.append(str(i))
print()


1
 step 30/30: ETA: 0:00:00 stepTimeAVG: 0.43829, Loss 0.100000 center_psnr: 36.34637  center_ssim: 0.95564  val_ssim: 0.89885  val_psnr: 30.80236  dof_ssim: 0.95526  dof_psnr: 35.55320  
